In [1]:
import pandas as pd

df_train = pd.read_csv(r'../data/train_tokenized.csv')
df_test = pd.read_csv(r'../data/test_tokenized.csv')

df_train

,post,gender,tokenized,cleaned,cluster
0,What? No escape? I thought that I would be ab...,1,"['escape', 'thought', 'would', 'able', 'get', ...","['escape', 'thought', 'able', 'need', 'hear', ...",7
1,"And all I can say is Thank You, Thank You!!! ...",1,"['say', 'thank', 'thank', 'think', 'look', 'an...","['thank', 'thank', 'think', 'look', 'surgical'...",7
2,She is a very very sweet girl :)\n\nIts hardly...,1,"['sweet', 'girl', 'hardly', '2', 'month', 'sin...","['sweet', 'girl', 'hardly', 'month', 'start', ...",5
3,"I love what Brooke had to say about the event,...",1,"['love', 'brooke', 'say', 'event', 'also', 'ti...","['love', 'brooke', 'event', 'tidbit', 'advice'...",2
4,I have some photos I've been meaning to post f...,1,"['photo', 'ive', 'mean', 'post', 'david', 'rei...","['photo', 'mean', 'post', 'david', 'reid', 'lo...",2
...,...,...,...,...,...
2569,"So, on my long run of the week I like to conte...",1,"['long', 'run', 'week', 'like', 'contemplate',...","['long', 'run', 'week', 'like', 'contemplate',...",7
2570,Contact lenses require a certain degree of car...,0,"['contact', 'lenses', 'require', 'certain', 'd...","['contact', 'lense', 'require', 'certain', 'de...",2
2571,I didn't start watching AD until way after the...,1,"['didnt', 'start', 'watch', 'ad', 'way', 'show...","['start', 'watch', 'way', 'late', 'know', 'beg...",2
2572,"At Creation Moments, Ian Taylor starts by aski...",0,"['creation', 'moment', 'ian', 'taylor', 'start...","['creation', 'moment', 'ian', 'taylor', 'start...",7


Note that male is encoded as 0, and female as 1.

In [2]:
df_test

,post,gender,tokenized,cleaned,cluster
0,perfect epitome of demagogue ..jus too senitiv...,1,"['perfect', 'epitome', 'demagogue', 'jus', 'se...","['perfect', 'epitome', 'demagogue', 'ju', 'sen...",5
1,Autistic Rights are Disability Rights are Huma...,1,"['autistic', 'right', 'disability', 'right', '...","['autistic', 'right', 'disability', 'right', '...",7
2,Good news for U.S. territories; Del. Donna M. ...,1,"['good', 'news', 'u', 'territories', 'del', 'd...","['good', 'news', 'territories', 'del', 'donna'...",6
3,The weather in this part of the globe is once ...,1,"['weather', 'part', 'globe', 'change', 'slight...","['weather', 'globe', 'change', 'slight', 'chil...",2
4,I was the first one to arrive at the station. ...,1,"['first', 'one', 'arrive', 'station', 'sit', '...","['arrive', 'station', 'sit', 'bench', 'wait', ...",2
...,...,...,...,...,...
639,"Sorry, Charlie\n\nUntil recently, it was assu...",0,"['sorry', 'charlie', 'recently', 'assume', 'ch...","['sorry', 'charlie', 'recently', 'assume', 'ch...",6
640,Feel Good….!!! :) Recently i have seen a video...,0,"['feel', 'good…', 'recently', 'see', 'video', ...","['feel', 'good', 'recently', 'video', 'abt', '...",5
641,"I've been working on some new material. "" When...",0,"['ive', 'work', 'new', 'material', 'youre', 'b...","['work', 'new', 'material', 'blind', 'sen', 's...",2
642,vasoooool raani :P.. finally here goes a testi...,0,"['vasoooool', 'raani', 'p', 'finally', 'go', '...","['vasoooool', 'raani', 'finally', 'testimonial...",5


In [3]:
df_train['tokenized'][0][0]

'['

In [4]:
from ast import literal_eval

df_train['tokenized'] = df_train['tokenized'].apply(literal_eval)
df_train['cleaned'] = df_train['cleaned'].apply(literal_eval)
df_test['tokenized'] = df_test['tokenized'].apply(literal_eval)
df_test['cleaned'] = df_test['cleaned'].apply(literal_eval)

In [5]:
df_train['tokenized'][0][0]

'escape'

### Load the BERT model and integrated gradients explainer

In [6]:
# imports
import numpy as np
import torch
import torch.nn.functional as nn
from transformers_interpret import SequenceClassificationExplainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import matplotlib.pyplot as plt

In [14]:
# load the BERT model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
pretrained_path = '../models/bert/own_script/epoch-4.model'
model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cpu')))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

<All keys matched successfully>

In [15]:
# load the explainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)

In [9]:
# functions to shorten the posts that are longer than BERT's max length
def truncate_post(post):
    truncated = post
    while len(tokenizer.encode(truncated)) > 512:
        truncated = ' '.join(truncated.split(' ')[:-1])
    return truncated

## Playing around with an example and some visualizations

In [65]:
test = "After lunch I need to finish my programming assignment."
exp = cls_explainer(test)

In [66]:
cls_explainer.visualize()
print(cls_explainer.predicted_class_index)
sorted(exp, key=lambda x: abs(x[1]), reverse=True)[:25]

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,LABEL_0 (0.62),LABEL_0,-0.07,[CLS] after lunch i need to finish my programming assignment . [SEP]


0


[('.', 0.5900083484474148),
 ('i', -0.5347733495175168),
 ('programming', -0.40283491858830833),
 ('after', 0.34998283854492707),
 ('lunch', 0.18180603598464137),
 ('need', -0.15578179248612883),
 ('my', -0.11040412193082169),
 ('to', 0.0884797107120352),
 ('finish', -0.058721041937792706),
 ('assignment', -0.018877573844514967),
 ('[CLS]', 0.0),
 ('[SEP]', 0.0)]

In [20]:
test = "After my lunch break, I need to finish my programming assignment."
exp = cls_explainer(test)

cls_explainer.visualize()
print(cls_explainer.predicted_class_index)
sorted(exp, key=lambda x: abs(x[1]), reverse=True)[:25]

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,LABEL_0 (0.66),LABEL_0,0.70,"[CLS] after my lunch break , i need to finish my programming assignment . [SEP]"


0


[('programming', 0.6231923814109792),
 ('after', 0.35921064063338065),
 ('.', 0.35607701024237964),
 ('i', -0.30119185871085363),
 ('lunch', -0.27147607516964833),
 ('to', 0.23823844957252152),
 ('my', -0.22830461115827214),
 ('my', -0.2018117935194161),
 ('need', 0.13375789746748845),
 (',', 0.11081922070666982),
 ('break', -0.10148490169532995),
 ('assignment', -0.0333175215745858),
 ('finish', 0.014229103182884515),
 ('[CLS]', 0.0),
 ('[SEP]', 0.0)]

# Apply LIME to UIC gender dataset

### On training data sample

In [67]:
n_samples = 5000
n_features = 25
idx = 2

print("Post: " + df_train.post[idx] + "\n")
print("Gender: " + str(df_train.gender[idx]) + "\n(0 = male, 1 = female)")
print("\nPost length: " + str(len(df_train.post[idx].split())))

instance = df_train.post[idx]
if len(tokenizer.encode(instance)) > 512:
    instance = truncate_post(instance)
    print("Truncated post. New post length: " + str(len(instance.split())))

Post: She is a very very sweet girl :)

Its hardly 2 months since I started talking to her. She is a fun loving girl. And she will make you smile :) Her smile is a most precious thing which she have. I always get inspired by way she overcome her stress.

Our IDS 506 gang just rocks!! We always meet to discuss about project but end up with guitar sessions or Marathi vocabulary or discussing about words like "Kote Rubber" or searching advertisements in you tube. In short, anything other than IDS 506 :D And what else you can except if Priya is around you!!

Keep Smiling Dear :) Hope you get everything in your life !!

Gender: 1
(0 = male, 1 = female)

Post length: 117


In [68]:
exp1 = cls_explainer(instance)
cls_explainer.visualize()
sorted(exp1, key=lambda x: abs(x[1]), reverse=True)[:20]

[('sweet', 0.373158325907526),
 ('girl', 0.34188213206495927),
 (':', 0.23648959883123968),
 (':', 0.1915660397744583),
 (':', 0.18264649107171452),
 ('she', 0.1777480268833769),
 ('.', 0.15625243818787155),
 ('loving', 0.1535577797171295),
 ('!', 0.1424758946969269),
 ('precious', 0.14044307522554658),
 ('her', 0.135434477990329),
 ('girl', 0.1349161953586425),
 ('##ya', 0.13452053987894347),
 ('!', 0.13433378092744333),
 ('fun', 0.13312032931222847),
 ('she', 0.13237440434404424),
 (':', 0.130739260826269),
 ('her', 0.1237288423451401),
 ('is', -0.12245645274510192),
 ('smile', 0.1191536210206266)]

### On test data

In [69]:
idx = 371
print("Post: " + df_test.post[idx] + "\n")
print("Gender: " + str(df_test.gender[idx]) + "\n(0 = male, 1 = female)")
print("\nPost length: " + str(len(df_test.post[idx].split())))

instance = df_test.post[idx]
if len(tokenizer.encode(instance)) > 512:
    instance = truncate_post(instance)
    print("Truncated post. New post length: " + str(len(instance.split())))

Post: No other country, with a history of successful education, applies economic principles of competition and incentives to the teaching profession. It’s too narrow-focused. There are so many other variables to consider. Community, family, individual differences. Schools are not factories. No other nation, with a successful school system, abandons courses in cultural subjects, like music and foreign languages, to focus only on basic skills and testing.

Gender: 0
(0 = male, 1 = female)

Post length: 65


In [70]:
exp2 = cls_explainer(instance)
cls_explainer.visualize()
sorted(exp2, key=lambda x: abs(x[1]), reverse=True)[:20]

[('so', 0.4483499303695508),
 ('economic', -0.34608749037337666),
 ('many', -0.31063114275570336),
 ('other', -0.2937264886059825),
 ('teaching', -0.2794551586718137),
 ('school', 0.21506281710761008),
 ('to', 0.1963066392132016),
 ('there', -0.18464145015619426),
 ('profession', 0.16331685950411054),
 ('schools', 0.15815050039534495),
 ('education', 0.15304241530995072),
 ('family', 0.13293977199257598),
 ('testing', -0.1241615596831525),
 ('abandon', 0.11500189563009922),
 ('.', -0.11430278296285924),
 (',', -0.1096584343866694),
 ('are', 0.0991990947927035),
 ('differences', -0.09911396447521026),
 (',', -0.09878989237368364),
 ('with', 0.08801944720674804)]

### Longer posts

In [71]:
idx = 100
print("Post: " + df_test.post[idx] + "\n")
print("Gender: " + str(df_test.gender[idx]) + "\n(0 = male, 1 = female)")
print("\nPost length: " + str(len(df_test.post[idx].split())))

instance = df_test.post[idx]
if len(tokenizer.encode(instance)) > 512:
    instance = truncate_post(instance)
    print("Truncated post. New post length: " + str(len(instance.split())))

Post: That's the second book in Timothy Zahn's trilogy.
I bet y'all thought that I'd forgotten about you concerning our fun trips. I realize that it's been a while since our vacation, but I might as well post the pictures that I was planning. The delay has been caused by our Village Missions conference and my presence here at Zachary and Brenda's house. Hopefully I'll be able to post some pictures of this fun soonish, but until then, without further ado, the unveiling of our Seaworld pictures.
I'm a big fan of the Orcas. They are much more majestic than their cousins: regulation orcs.
I was pretty sure, beforehand, that whaling was illegal in most states these days, but I certainly enjoyed my portion.
There's a likelihood that I was a bigger fan of the Belugas than of anything else. For whatever reason, the all-white whales (or the all-black Pilot Whales which I have no pictures of) just greatly entertain me. Perchance my monochromatic preferences reflect my worldview?
Here's a good pi

In [72]:
exp3 = cls_explainer(instance)
cls_explainer.visualize()
sorted(exp3, key=lambda x: abs(x[1]), reverse=True)[:20]

[('zachary', 0.4595006178345882),
 ('or', 0.2295855292997598),
 ('whaling', 0.21507486818754973),
 ('brenda', 0.21110111544261936),
 ('whales', 0.18568800336485303),
 ('regulation', -0.17483664942441396),
 ('said', -0.1746611280968344),
 ('conference', -0.1618154057447191),
 ('timothy', 0.1568501505123864),
 ('states', 0.15583322421322668),
 ('by', -0.14614626538424474),
 ('s', -0.14345811685838675),
 ('##cas', 0.13870641638554373),
 ('.', -0.13169323675148856),
 (':', 0.12732756929392092),
 ('y', -0.11697641776343785),
 ('her', 0.11522159552144869),
 ('whales', 0.11222830674977835),
 ('of', -0.10550643774565728),
 ('i', -0.10336912769319745)]

In [73]:
idx = 640
print("Post: " + df_test.post[idx] + "\n")
print("Gender: " + str(df_test.gender[idx]) + "\n(0 = male, 1 = female)")
print("\nPost length: " + str(len(df_test.post[idx].split())))

instance = df_test.post[idx]
if len(tokenizer.encode(instance)) > 512:
    instance = truncate_post(instance)
    print("Truncated post. New post length: " + str(len(instance.split())))

Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


Post: Feel Good….!!! :) Recently i have seen a video which says abt the secret of life.... tat video has inspired me a lot..
This is wat all of the video is about...

One of the best feelings that one can enjoy is to Feel Good. Feeling Good gives all the good feelings one can experience in life...either could be .... happiness, joy, love, gratitude, Appreciation, optimism, belief, hope, contentment, passion... watever
Feeling Good makes oneself to aspire watever he or she wants to be.... :) it involves a lot of positive energy in it ... to make tat happen...
These are the toughest times the world is facing now.... the terrorism, the financial crisis... or anything which disrupts peace...presently the world is ever bounding with all forms of negative feeling.. either could be greed, guilt, revenge, jealous , hatred, frustration .. these forms of feelings in oneself is directly or indirectly responsible for the present situation of the world... so let all of us stop blaming others ... st

In [74]:
exp4 = cls_explainer(instance)
cls_explainer.visualize()
sorted(exp4, key=lambda x: abs(x[1]), reverse=True)[:20]

[('society', -0.4758688660713773),
 ('financial', -0.47062677978010636),
 ('crisis', 0.3504884433768005),
 ('a', -0.23896528572831338),
 ('i', 0.23280652450809947),
 ('recently', 0.13159321199174026),
 (',', 0.1290155922830224),
 ('.', -0.1167327228908287),
 ('health', -0.11419538377961783),
 ('.', -0.11189290422709477),
 ('terrorism', -0.10033059080734978),
 ('.', 0.08887638821775186),
 ('the', 0.0821406010445763),
 (',', -0.07964020077882082),
 ('the', -0.07765128678468843),
 ('in', 0.07654111669533502),
 ('the', 0.07642468189407696),
 ('terrorism', -0.07585777699957172),
 ('passion', 0.07179607897212867),
 ('.', -0.07093660054698926)]

In [75]:
idx = 348
print("Post: " + df_test.post[idx] + "\n")
print("Gender: " + str(df_test.gender[idx]) + "\n(0 = male, 1 = female)")
print("\nPost length: " + str(len(df_test.post[idx].split())))

instance = df_test.post[idx]
if len(tokenizer.encode(instance)) > 512:
    instance = truncate_post(instance)
    print("Truncated post. New post length: " + str(len(instance.split())))

Post: You know life is life you can’t really chose where it takes you, god has a reason for everything that is the way I look at it. Sure you going to lose those you think you love and those you do, but if it wasn’t meant to be, there is nothing you can do but love, live and go on about your life. Sometime you get to the point when you have to just stop and think about all the positive things that have happened to you and stop thinking about the things in your life that you have fucked up, or things that have happened to you. My life has taken many many turn's some for the good some for the bad, but I am who I am from my past. I have many people that have been there for me throw everything and some that are just there to support me when I need someone to talk to, I am very thankful for those people and you know who you are. But I have to thank my dad for most of it, we have had are times like may may kids but he has never giving up on me even sometimes when he wanted to just drop kick 

In [76]:
exp5 = cls_explainer(instance)
cls_explainer.visualize()
sorted(exp5, key=lambda x: abs(x[1]), reverse=True)[:20]

[('dad', 0.46821974290509216),
 ('fuck', -0.46717061579637553),
 ('just', -0.26720897156261614),
 ('##in', 0.26419498716311),
 ('it', 0.2603010119981949),
 ('over', -0.15584609845809846),
 ('can', 0.15093821066250185),
 ('love', 0.1337603466933085),
 ('and', -0.13207790209422696),
 ('!', 0.11801725192331411),
 ('today', 0.11168165291736294),
 ('love', 0.10201514061264619),
 ('for', 0.09861028872946014),
 ('those', 0.09383937324201472),
 ('those', 0.0933775710438527),
 (',', -0.09220487628250801),
 ('am', -0.08470699415512604),
 ('my', 0.08371294577973246),
 ('that', -0.07874873786573491),
 ('fucked', -0.07581558533042736)]

### Explanations for a selected nr. of samples

In [10]:
explanation_df = pd.read_csv(r'../data/explanation_df.csv')
explanation_df

,df_test_index,post,label,lime,ig
0,3,The weather in this part of the globe is once ...,1,NaN,NaN
1,6,sometimes in life we get so excited about what...,1,NaN,NaN
2,7,I had a wonderful encounter with God yesterday...,1,NaN,NaN
3,9,"Did not win on Friday. In fact, only got one d...",0,NaN,NaN
4,13,Meg's recent post on fears and dreams in partn...,1,NaN,NaN
...,...,...,...,...,...
195,628,"Ideally, the process for buying a laptop would...",0,NaN,NaN
196,630,The big star of today's Apple event in San Fra...,0,NaN,NaN
197,632,Well it's that time again! Time for me to com...,0,NaN,NaN
198,633,USAA automobile insurance was initially for mi...,1,NaN,NaN


In [11]:
explanation_df['ig'] = explanation_df['ig'].astype('object')
explanation_df.dtypes

df_test_index      int64
post              object
label              int64
lime             float64
ig                object
dtype: object

In [12]:
for index, row in explanation_df.iterrows():
    print(index)
    instance = row['post']
    if len(tokenizer.encode(instance)) > 512:
        instance = truncate_post(instance)
        print("Post exceeded max length of BERT -> truncated the post.")
    exp = cls_explainer(instance)
    explanation_df.at[index,'ig'] = exp

0
1


Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors


2
Post exceeded max length of BERT -> truncated the post.
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Post exceeded max length of BERT -> truncated the post.
26
27
28
29
30
31
32
33
Post exceeded max length of BERT -> truncated the post.
34
35
36
37
Post exceeded max length of BERT -> truncated the post.
38
39
40
41
42
Post exceeded max length of BERT -> truncated the post.
43
44
45
46
47
48
49
50
51
52
Post exceeded max length of BERT -> truncated the post.
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
Post exceeded max length of BERT -> truncated the post.
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
Post exceeded max length of BERT -> truncated the post.
121
122
123
124
125
126
Post exceeded max length of BERT -> truncated the post.
127
128
129
Post exceeded max length of BERT -> truncated the post.
130
131
132
133
134
135
136
Post e

In [13]:
explanation_df

,df_test_index,post,label,lime,ig
0,3,The weather in this part of the globe is once ...,1,NaN,"[([CLS], 0.0), (the, -0.03305793469461231), (w..."
1,6,sometimes in life we get so excited about what...,1,NaN,"[([CLS], 0.0), (sometimes, -0.0027479165931052..."
2,7,I had a wonderful encounter with God yesterday...,1,NaN,"[([CLS], 0.0), (i, 0.08986844054184116), (had,..."
3,9,"Did not win on Friday. In fact, only got one d...",0,NaN,"[([CLS], 0.0), (did, -0.07365760226623197), (n..."
4,13,Meg's recent post on fears and dreams in partn...,1,NaN,"[([CLS], 0.0), (meg, 0.18647030206724557), (',..."
...,...,...,...,...,...
195,628,"Ideally, the process for buying a laptop would...",0,NaN,"[([CLS], 0.0), (ideally, 0.02353497014460397),..."
196,630,The big star of today's Apple event in San Fra...,0,NaN,"[([CLS], 0.0), (the, 0.1460338751768401), (big..."
197,632,Well it's that time again! Time for me to com...,0,NaN,"[([CLS], 0.0), (well, -0.0063752897228923685),..."
198,633,USAA automobile insurance was initially for mi...,1,NaN,"[([CLS], 0.0), (usa, 0.011359380673091573), (#..."


In [16]:
sorted(explanation_df.ig[0], key=lambda x: abs(x[1]), reverse=True)[:25]

[('philippine', 0.867071641373719),
 ('globe', 0.1982692565557989),
 ('of', -0.18361505095208377),
 ('our', 0.17942234230365517),
 ('.', -0.14111155595504893),
 ('to', -0.10372748037865072),
 ('the', 0.10276892341468986),
 ('summer', -0.09096910679529133),
 ('time', 0.08079262658097039),
 ('tropical', 0.08047230363280118),
 ('rainy', 0.07612957567542894),
 ('warmer', -0.07504075697358473),
 ('has', 0.06816352441646152),
 ('really', 0.06581046439116311),
 ('once', -0.06345105425196736),
 ('weather', -0.0602213800031324),
 (',', -0.058607082363393063),
 ('it', -0.05820517429097139),
 ('because', -0.05106827557618718),
 ('shift', 0.050834243258093305),
 ('be', -0.05010573761102577),
 ('would', -0.04856899631814525),
 ('expected', 0.047571991182854254),
 ('.', -0.04647747238756581),
 ('entering', 0.04550527354913574)]

### Store explanations

In [14]:
from operator import index

explanation_df.to_csv('../data/explanation_df.csv', index=False)

### All test data samples

In [35]:
full_explanation_df = pd.read_csv(r'../data/full_explanation_df.csv')
full_explanation_df

,post,label,lime,ig,truncated,pred,ig_accuracy,lime_accuracy
0,perfect epitome of demagogue ..jus too senitiv...,1,"[('cream', 0.15870011789516264), ('n', 0.15779...","[('[CLS]', 0.0), ('perfect', 0.050485146477939...",perfect epitome of demagogue ..jus too senitiv...,1,0.098951,0.078889
1,Autistic Rights are Disability Rights are Huma...,1,"[('I', 0.12863655638063162), ('love', 0.119193...","[('[CLS]', 0.0), ('au', 0.05108341832547773), ...",Autistic Rights are Disability Rights are Huma...,1,0.035142,0.061481
2,Good news for U.S. territories; Del. Donna M. ...,1,"[('women', 0.13086390728973565), ('her', 0.093...","[('[CLS]', 0.0), ('good', 0.11303952297093836)...",Good news for U.S. territories; Del. Donna M. ...,1,0.176900,0.313036
3,The weather in this part of the globe is once ...,1,"[('summer', 0.07622637896033177), ('and', 0.07...","[('[CLS]', 0.0), ('the', -0.03305793469461231)...",The weather in this part of the globe is once ...,1,0.452370,0.397931
4,I was the first one to arrive at the station. ...,1,"[('Marylou', 0.23855737996628787), ('I', 0.135...","[('[CLS]', 0.0), ('i', 0.15665139217651933), (...",I was the first one to arrive at the station. ...,1,0.048063,0.111751
...,...,...,...,...,...,...,...,...
639,"Sorry, Charlie\n\nUntil recently, it was assu...",0,"[('Rubio', 0.008505039532604477), ('GOP', 0.00...","[('[CLS]', 0.0), ('sorry', 0.00548099766662814...","Sorry, Charlie\n\nUntil recently, it was assu...",0,-0.001063,-0.010613
640,Feel Good….!!! :) Recently i have seen a video...,0,"[('terrorism', -0.036446640999250664), ('love'...","[('[CLS]', 0.0), ('feel', 0.019222500626415676...",Feel Good….!!! :) Recently i have seen a video...,0,-0.019081,-0.004429
641,"I've been working on some new material. "" When...",0,"[('I', 0.14368967312370054), ('Alter', -0.1385...","[('[CLS]', 0.0), ('i', 0.04488389842466153), (...","I've been working on some new material. "" When...",0,0.018149,-0.009480
642,vasoooool raani :P.. finally here goes a testi...,0,"[('staff', -0.06092283569439009), ('felicitati...","[('[CLS]', 0.0), ('va', 0.03454713593228292), ...",vasoooool raani :P.. finally here goes a testi...,0,0.168383,-0.027275


In [32]:
full_explanation_df['ig'] = full_explanation_df['ig'].astype('object')

for index, row in full_explanation_df.iterrows():
    if index%10 == 0:
        print(index)
    instance = row['truncated']
    exp = cls_explainer(instance)
    full_explanation_df.at[index,'ig'] = exp

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440


In [33]:
from operator import index

full_explanation_df.to_csv('../data/full_explanation_df.csv', index=False)